In [1]:
import json
import pandas as pd 
import os
import cv2

In [2]:
odtg_file_path = 'C:\\Users\\Hallvard\\Code\\dataset\\CrowdHuman\\annotation_all.odgt'

# Read and immediately convert each line to a Python dict to avoid reading the entire file into memory at once
with open(odtg_file_path, 'r') as f:
    data = [json.loads(line) for line in f]

# Convert data to DataFrame
df_gt = pd.DataFrame(data)

# Rename 'ID' column for consistency
df_gt = df_gt.rename(columns={'ID': 'image_id'})

# Explode the 'gtboxes' column to separate rows and normalize
df_exploded = df_gt.explode('gtboxes')
df_gtboxes = pd.json_normalize(df_exploded['gtboxes'])

# Concatenate 'image_id' with normalized 'gtboxes' DataFrame
df_gt = pd.concat([df_exploded[['image_id']].reset_index(drop=True), df_gtboxes], axis=1)

# Filter for 'person' tags and ignore == 0 or NaN
df_gt = df_gt[(df_gt['tag'] == 'person') & (df_gt['extra.ignore'].isna() | (df_gt['extra.ignore'] == 0))]

# Drop unnecessary columns in one step to avoid multiple DataFrame traversals
to_drop = ['tag', 'hbox', 'fbox', 'head_attr.ignore', 'head_attr.occ', 'head_attr.unsure', 
           'extra.box_id', 'extra.occ', 'extra.ignore', 'extra.unsure']
df_gt.drop(to_drop, axis=1, inplace=True)

# Direct conversion of 'vbox' lists to DataFrame columns
vbox_columns = ['x', 'y', 'w', 'h']
df_vbox = pd.DataFrame(df_gt.pop('vbox').tolist(), columns=vbox_columns)

# Final DataFrame combining 'image_id' with 'vbox' columns
df_gt_xywh = pd.concat([df_gt.reset_index(drop=True), df_vbox], axis=1)

In [3]:
image_path = 'C:\\Users\\Hallvard\\Code\\dataset\\CrowdHuman\\CrowdHuman-yolo\\images'

# List comprehension to build image data
def create_image_shapes_df(image_path):
    image_data = [
        (os.path.splitext(file_name)[0], img.shape[1], img.shape[0])
        for file_name in os.listdir(image_path)
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg'))
        if (img := cv2.imread(os.path.join(image_path, file_name))) is not None
    ]
    
    # Create DataFrame
    df_image_shapes = pd.DataFrame(image_data, columns=['image_id', 'image_width', 'image_height'])
    return df_image_shapes
df_shapes = create_image_shapes_df(image_path)

In [6]:
# Merge df_shapes and df_gt_xywh on image_id
df_gt_xywh_shapes = pd.merge(df_gt_xywh, df_shapes, on='image_id', how='inner')

In [8]:
def convert_to_xywh(df):
    # Centralized instead of upper left, and normalize the values.
    # Calculate non-normalized width and height
    df = df.assign(
        x_center_norm=(df['x']+(df['w']/2))/df['image_width'],
        y_center_norm=(df['y']+(df['h']/2))/df['image_height'],
        w_norm=df['w']/df['image_width'],
        h_norm=df['h']/df['image_height']
    )
    
    # Drop the previous 
    df.drop(['x', 'y', 'w', 'h', 'image_width', 'image_height'], axis=1, inplace=True)

    return df

In [10]:
df = convert_to_xywh(df_gt_xywh_shapes)

In [11]:
df.head(5)

,image_id,x_center_norm,y_center_norm,w_norm,h_norm
0,"273271,1a0d6000b9e1f5b7",0.919922,0.577901,0.162109,0.338633
1,"273271,1a0d6000b9e1f5b7",0.774414,0.539746,0.117188,0.392687
2,"273271,1a0d6000b9e1f5b7",0.650391,0.494436,0.113281,0.486486
3,"273271,1a0d6000b9e1f5b7",0.467285,0.477742,0.110352,0.462639
4,"273271,1a0d6000b9e1f5b7",0.577148,0.416534,0.125000,0.505564


In [22]:
output_folder = 'C:\\Users\\Hallvard\\Code\\dataset\\CrowdHuman\\CrowdHuman-yolo\\labels'
# Ensure output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

lines_written = 0

# Process each entry in the JSON
for bbox in df.values:
    filename = os.path.join(output_folder, f"{bbox[0]}.txt")
    with open(filename, 'a') as output_file:
        lines_written += 1
        output_file.write(f"0 {bbox[1]} {bbox[2]} {bbox[3]} {bbox[4]}\n")
    if(lines_written % 100 == 0):
        print(f'{lines_written} lines written.')
print(f'{lines_written} lines written.')


57 lines written.
